<a href="https://colab.research.google.com/github/vamfish/hello-world/blob/master/Stable_Diffusion_WebUI_Colab_TW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion WebUI Colab TW

### 使用流程

1. 到**設置模型**及**進階設置**進行設置
2. 左上角工具列 > 執行階段 > 變更執行階段類型 > 硬體加速器改為 GPU
3. 左上角工具列 > 執行階段 > 全部執行
4. 到**啟動 WebUI** 點選 public URL

### 巴哈原文
https://home.gamer.com.tw/artwork.php?sn=5664550

### 參考資料
https://github.com/ddPn08/automatic1111-colab

## 1 - 下載 stable-diffusion-webui

In [ ]:
%cd /content/
repository_url = "https://github.com/AUTOMATIC1111/stable-diffusion-webui"  # @param {type: "string"}
webui_branch = "master"  # @param {type: "string"}

! git clone {repository_url}
%cd /content/stable-diffusion-webui
! git checkout {webui_branch}

## 2 - 設置模型

In [ ]:
# @title 設置SD模型
%cd /content
# @markdown 雲端資料夾名稱
data_dir_name = "Stable_Diffusion_WebUI_Colab_TW"  # @param {type:"string"}
data_dir = f"/content/drive/MyDrive/{data_dir_name}"
# @markdown 選擇SD模型組合包
model_package = "AnythingV4.5" # @param ["AnythingV3", "AnythingV4.5", "AbyssOrangeMix2_sfw", "AbyssOrangeMix3", "ChilloutMix", "其他(下方填寫)"] 
# @markdown 其他SD模型網址
sd_url = "" #@param {type:"string"}
vae_url = "" #@param {type:"string"}
# @markdown ---
# @markdown - VAE 開啟後圖片色彩比較豐富<br>
# @markdown  - WebUI > Settings > Stable Diffusion > SD VAE 設定<br>
# @markdown  - 記得要按下 Apply settings

sd_urls = []
vae_urls = []
if model_package == "AnythingV3":
    sd_urls.append("https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp32-pruned.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/my_vae/resolve/main/Anything-V3.0.vae.pt")
elif model_package == "AnythingV4.5":
    sd_urls.append("https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.safetensors")
    vae_urls.append("https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt")
elif model_package == "AbyssOrangeMix2_sfw":
    sd_urls.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_sfw.safetensors")
    vae_urls.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt")
elif model_package == "AbyssOrangeMix3":
    sd_urls.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3.safetensors")
    vae_urls.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt")
elif model_package == "ChilloutMix":
    sd_urls.append("https://huggingface.co/WuLing/Chilloutmix_package/resolve/main/chilloutmix_NiPrunedFp32.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/Chilloutmix_package/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors")
elif model_package == "其他(下方填寫)":
    sd_urls.append(sd_url)
    vae_urls.append(vae_url)

import os

drive_path = "/content/drive"
from google.colab import drive

try:
    drive.mount(drive_path, force_remount=True)
    os.makedirs(data_dir, exist_ok=True)
except:
    print('Mount Error')
    raise

if os.path.exists(f"{data_dir}/script.pre.sh"):
    ! chmod +x {data_dir}/script.pre.sh
    ! {data_dir}/script.pre.sh

models_path = f"{data_dir}/models"
output_path = f"{data_dir}/outputs"
config_path = f"{data_dir}/config"
scripts_path = f"{data_dir}/scripts"
extensions_file_path = f"{data_dir}/extensions.txt"

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(config_path, exist_ok=True)
os.makedirs(scripts_path, exist_ok=True)
os.makedirs(f"{models_path}/Stable-diffusion", exist_ok=True)
os.makedirs(f"{models_path}/VAE", exist_ok=True)
os.makedirs(f"{models_path}/embeddings", exist_ok=True)
os.makedirs(f"{models_path}/hypernetworks", exist_ok=True)
os.makedirs(f"{models_path}/Lora", exist_ok=True)

for script in os.listdir(scripts_path):
    ! rm -Rf stable-diffusion-webui/scripts/{script} && ln -s {scripts_path}/{script} stable-diffusion-webui/scripts/{script}

for dir in os.listdir(models_path):
    if dir == "embeddings":
        ! rm -Rf stable-diffusion-webui/embeddings && ln -s {models_path}/embeddings stable-diffusion-webui/embeddings
    else:
        ! rm -Rf stable-diffusion-webui/models/{dir} && ln -s {models_path}/{dir} stable-diffusion-webui/models/{dir}

! rm -Rf stable-diffusion-webui/outputs && ln -s {data_dir}/outputs stable-diffusion-webui/outputs

for filename in ["config.json", "ui-config.json", "styles.csv", "artists.csv"]:
    ! rm -f stable-diffusion-webui/{filename}
    filepath = f"{config_path}/{filename}"
    if not os.path.exists(filepath):
        if filename.endswith(".json"):
            with open(filepath, mode="w") as f:
                f.write("{}")
        else:
            ! touch {config_path}/{filename}
    ! ln -s {config_path}/{filename} stable-diffusion-webui/{filename}

if sd_urls != []:
    %cd {models_path}/Stable-diffusion
    for sd_url in sd_urls:
        if sd_url != "":
            ! wget -nc --content-disposition {sd_url}

if vae_urls != []:
    %cd {models_path}/VAE
    for vae_url in vae_urls:
        if vae_url != "":  
            ! wget -nc --content-disposition {vae_url}

In [ ]:
# @title 設置 Embedding + Hypernetwork + LoRA
# @markdown Embedding
ulzzang = False # @param {type:"boolean"}
embedding_urls = ""  # @param {type:"string"}
# @markdown ---
# @markdown Hypernetwork
hypernetwork_urls = ""  # @param {type:"string"}
# @markdown ---
# @markdown LoRA
korean_doll_likeness = False # @param {type:"boolean"}
taiwan_doll_likeness = False # @param {type:"boolean"}
japanese_doll_likeness = False # @param {type:"boolean"}
lora_urls = ""  # @param {type:"string"}
# @markdown ---
# @markdown - ulzzang embedding 的觸發詞為 ulzzang<br>
# @markdown - doll_likeness系列的權重建議為0.5<br>
# @markdown - 網址之間用英文逗號隔開<br>

embeddings = []
embedding_urls = list(map(str.strip, embedding_urls.split(",")))
if ulzzang == True:
    embeddings.append("https://civitai.com/api/download/models/10107")
if embedding_urls != ['']:
    embeddings = embeddings + embedding_urls

if embeddings != []:
    %cd {models_path}/embeddings
    for embedding in embeddings:
        ! wget -nc --content-disposition {embedding}

hypernetworks = []
hypernetwork_urls = list(map(str.strip, hypernetwork_urls.split(",")))
if hypernetwork_urls != ['']:
    hypernetworks = hypernetworks + hypernetwork_urls

if hypernetworks != []:
    %cd {models_path}/hypernetworks
    for hypernetwork in hypernetworks:
        ! wget -nc --content-disposition {hypernetwork}

loras = []
lora_urls = list(map(str.strip, lora_urls.split(",")))
if korean_doll_likeness == True:
    loras.append("https://civitai.com/api/download/models/8750")
if taiwan_doll_likeness == True:
    loras.append("https://civitai.com/api/download/models/9070")
if japanese_doll_likeness == True:
    loras.append("https://civitai.com/api/download/models/12050")
if lora_urls != ['']:
    loras = loras + lora_urls

if loras != []:
    %cd {models_path}/Lora
    for lora in loras:
        ! wget -nc --content-disposition {lora}

## 3 - 進階設置

In [ ]:
# @title 2.1 命令行參數
import os

os.environ["COMMANDLINE_ARGS"] = "--no-half-vae --xformers --share --gradio-queue --theme dark"

In [ ]:
# @title 2.2 Extensions
tag_complete = True # @param {type:"boolean"}
control_net = False # @param {type:"boolean"}
extension_urls = ""  # @param {type:"string"}
# @markdown ---
# @markdown - tag_complete 提供 Danbooru tag 建議
# @markdown - control_net 提供控制姿勢等方法
# @markdown - 網址之間用英文逗號隔開<br>

extensions = []
extension_urls = list(map(str.strip, extension_urls.split(",")))
if tag_complete == True:
    extensions.append("https://github.com/DominikDoom/a1111-sd-webui-tagcomplete")
if control_net == True:
    extensions.append("https://github.com/Mikubill/sd-webui-controlnet")
if extension_urls != ['']:
    extensions = extensions + extension_urls

import os

if extensions != []:
    %cd /content/stable-diffusion-webui/extensions
    for extension in extensions:
        if extension.startswith("#"):
            continue
        ! git clone {extension}
        extension_name, _ = os.path.splitext(extension.split("/")[-1])
        if not os.path.isdir(extension_name):
            ! git clone {extension}
        else:
            ! cd {extension_name} && git fetch

In [ ]:
# @title 2.3 ControlNet 模型導入
control_canny = False # @param {type:"boolean"}
control_depth = False # @param {type:"boolean"}
control_hed = False # @param {type:"boolean"}
control_mlsd = False # @param {type:"boolean"}
control_normal = False # @param {type:"boolean"}
control_openpose = True # @param {type:"boolean"}
control_scribble = False # @param {type:"boolean"}
control_seg = False # @param {type:"boolean"}
t2iadapter_keypose = False # @param {type:"boolean"}
t2iadapter_seg = False # @param {type:"boolean"}
t2iadapter_sketch = False # @param {type:"boolean"}
# @markdown ---
# @markdown - 參考資料
# @markdown  - https://github.com/lllyasviel/ControlNet
# @markdown  - https://github.com/Mikubill/sd-webui-controlnet
import os

control_net_models = []
if control_canny == True:
    control_net_models.append("control_canny")
if control_depth == True:
    control_net_models.append("control_depth")
if control_hed == True:
    control_net_models.append("control_hed")
if control_mlsd == True:
    control_net_models.append("control_mlsd")
if control_normal == True:
    control_net_models.append("control_normal")
if control_openpose == True:
    control_net_models.append("control_openpose")
if control_scribble == True:
    control_net_models.append("control_scribble")
if control_seg == True:
    control_net_models.append("control_seg")
if t2iadapter_keypose == True:
    control_net_models.append("t2iadapter_keypose")
if t2iadapter_seg == True:
    control_net_models.append("t2iadapter_seg")
if t2iadapter_sketch == True:
    control_net_models.append("t2iadapter_sketch")

%cd /content/stable-diffusion-webui/extensions

if os.path.exists('sd-webui-controlnet'):
    %cd sd-webui-controlnet/models
    for control_net_model in control_net_models:
        ! wget -nc --content-disposition https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/{control_net_model}-fp16.safetensors

## 4 - 建立環境

In [ ]:
%cd /content

conda_dir = "/opt/conda"
conda_bin = os.path.join(conda_dir, "bin", "conda")

if not os.path.exists(conda_bin):
    ! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    ! chmod +x Miniconda3-latest-Linux-x86_64.sh
    ! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p {conda_dir}
    ! rm Miniconda3-latest-Linux-x86_64.sh


install_script = f"""
eval "$({conda_bin} shell.bash hook)"
cd stable-diffusion-webui
python3 -m pip install --upgrade tensorrt triton
python -c 'from launch import prepare_environment; prepare_environment()'
pip install --upgrade fastapi==0.90.1
"""

! {install_script}

patches_dir = "/content/patches"
os.makedirs(patches_dir, exist_ok=True)
! cd {patches_dir} && curl -LO https://raw.githubusercontent.com/ddPn08/automatic1111-colab/main/patches/stablediffusion-lowram.patch
! cd /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai && git apply {patches_dir}/stablediffusion-lowram.patch

os.environ[
    "LD_LIBRARY_PATH"
] = f"{os.environ['LD_LIBRARY_PATH']}:/usr/local/envs/automatic/lib"

if os.path.exists(f"{data_dir}/script.post.sh"):
    ! chmod +x {data_dir}/script.post.sh
    ! {data_dir}/script.post.sh

## 5 - 啟動 WebUI

In [ ]:
# @markdown - 點選 public URL 開啟 WebUI<br>
# @markdown ![](https://i.imgur.com/y3xGiIX.png)

%cd /content/stable-diffusion-webui/
! git pull

run_script = f"""
eval "$({conda_bin} shell.bash hook)"
accelerate launch --num_cpu_threads_per_process 1 launch.py
"""
! {run_script}